In [1]:
import pandas as pd
from Transaction import *
from nacl.signing import SigningKey
from nacl.encoding import Base64Encoder
from hashlib import sha256
from nacl.encoding import HexEncoder

In [2]:
# create the keys first using a dataframe 
keys_list = pd.DataFrame()

In [4]:
# generate a list of 8 public keys and 8 private keys along with them and the signatures along with them since the signatures should stay the same for each public key
pub_keys_list = []
secret_keys_list = []
users_list = []
user = 'user'
for x in range(0, 8):
    users_list.append(user + str(x))
    sk = SigningKey.generate()
    secret_keys_list.append(sk)
    pk = sk.verify_key
    pub_keys_list.append(pk)

In [5]:
# add these two list to the dataframe
keys_list['users'] = users_list
keys_list['pubkey'] = pub_keys_list
keys_list['sk'] = secret_keys_list

In [6]:
from nacl.signing import SigningKey

def createSignature(input, output, user_sk):
    # temp = input.encode()
    temp = output.encode()
    signature = user_sk.sign(temp, encoder=Base64Encoder).decode()
    return signature

In [15]:
# create the genesis transaction first - create 8 different ones 
output_1 = transaction_output("100", keys_list['pubkey'][0].encode(encoder=Base64Encoder).decode())
output_2 = transaction_output("100", keys_list['pubkey'][1].encode(encoder=Base64Encoder).decode())
output_3 = transaction_output("100", keys_list['pubkey'][2].encode(encoder=Base64Encoder).decode())
output_4 = transaction_output("100", keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode())
output_5 = transaction_output("100", keys_list['pubkey'][4].encode(encoder=Base64Encoder).decode())
output_6 = transaction_output("100", keys_list['pubkey'][5].encode(encoder=Base64Encoder).decode())
output_7 = transaction_output("100", keys_list['pubkey'][6].encode(encoder=Base64Encoder).decode())
output_8 = transaction_output("100", keys_list['pubkey'][7].encode(encoder=Base64Encoder).decode())

In [17]:
all_outputs = []
all_outputs.append(output_1)
all_outputs.append(output_2)
all_outputs.append(output_3)
all_outputs.append(output_4)
all_outputs.append(output_5)
all_outputs.append(output_6)
all_outputs.append(output_7)
all_outputs.append(output_8)

In [18]:
g_sig = createSignature(None, json.dumps([o.to_dict() for o in all_outputs]), keys_list['sk'][0])

In [19]:
g_transaction = Transaction([], all_outputs, g_sig, None, True)
json_object = json.dumps(g_transaction.to_dict(), indent=4)
with open("transactions/genesis_tx.json", "w") as outfile:
    outfile.write(json_object)

# Create file for double spending 

In [11]:
from nacl.signing import SigningKey

def createSignature_nonGenesis(input, output, user_sk):
    temp = input.encode()
    temp += output.encode()
    signature = user_sk.sign(temp, encoder=Base64Encoder).decode()
    return signature

In [20]:
ds_input_tx = transaction_input("1", output_1, None)
ds_output_tx1 = transaction_output("50", keys_list['pubkey'][0].encode(encoder=Base64Encoder).decode(), None)
ds_output_tx2 = transaction_output("50", keys_list['pubkey'][1].encode(encoder=Base64Encoder).decode(), None)
ds_input = []
ds_input.append(ds_input_tx)
ds_output = []
ds_output.append(ds_output_tx1)
ds_output.append(ds_output_tx2)
ds_sig = createSignature_nonGenesis(json.dumps(([i.to_dict() for i in ds_input])),json.dumps([o.to_dict() for o in ds_output]), keys_list['sk'][0])
ds_1_Transaction = Transaction(ds_input, ds_output, ds_sig, None, False)

2nd Transaction for Double Spend 

In [21]:
second_ds_input_tx = transaction_input("1", output_1, None)
second_ds_output_tx1 = transaction_output(50, keys_list['pubkey'][2].encode(encoder=Base64Encoder).decode(), None)
second_ds_output_tx2 = transaction_output(50, keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), None)
ds_total_input = []
ds_total_input.append(second_ds_input_tx)
ds_total_output = []
ds_total_output.append(second_ds_output_tx1)
ds_total_output.append(second_ds_output_tx2)
ds_sig = createSignature_nonGenesis(json.dumps(([i.to_dict() for i in ds_total_input])),json.dumps([o.to_dict() for o in ds_total_output]), keys_list['sk'][0])
ds_2_Transaction = Transaction(ds_total_input, ds_total_output, ds_sig, None, False)

In [23]:
json_object1 = ds_1_Transaction.to_dict()
json_object2 = ds_2_Transaction.to_dict()
list_objects = []
list_objects.append(json_object1)
list_objects.append(json_object2)
json_str = json.dumps(list_objects, indent=4)
with open("transactions/double_spend.json", "w") as outfile:
    outfile.write(json_str)


Input and Output Match Testfile 

In [111]:
sum_input_1 = transaction_input('5c777165b6968cd976b25a791bd7a2ce2746b0579ff3af0575cb41b96711b674', ds_output_tx1, None)
sum_output_1 = transaction_output("25", keys_list['pubkey'][0].encode(encoder=Base64Encoder).decode(), [])
sum_output_2 = transaction_output("25", keys_list['pubkey'][7].encode(encoder=Base64Encoder).decode(), [])
total_sum_input = []
total_sum_input.append(sum_input_1)
total_sum_output = []
total_sum_output.append(sum_output_1)
total_sum_output.append(sum_output_2)
sum_sig = createSignature_nonGenesis(json.dumps(([i.to_dict() for i in total_sum_input])),json.dumps([o.to_dict() for o in total_sum_output]), keys_list['sk'][0])
sum_1_Transaction = Transaction([], total_sum_input, total_sum_output, sum_sig, None, 0)

Create second transaction 

In [118]:
sum_input_2 = transaction_input(1, output_2, None)
sum_input_3 = transaction_input("5c777165b6968cd976b25a791bd7a2ce2746b0579ff3af0575cb41b96711b674", ds_output_tx2, None)
sum_output_3 = transaction_output(100, keys_list['pubkey'][4].encode(encoder=Base64Encoder).decode(), [])
sum_output_4 = transaction_output(40, keys_list['pubkey'][7].encode(encoder=Base64Encoder).decode(), [])
total_sum_input_1 = []
total_sum_output_1 = []
total_sum_input_1.append(sum_input_2)
total_sum_input_1.append(sum_input_3)
total_sum_output_1.append(sum_output_3)
total_sum_output_1.append(sum_output_4)
second_sum_sig = createSignature_nonGenesis(json.dumps(([i.to_dict() for i in total_sum_input_1])),json.dumps([o.to_dict() for o in total_sum_output_1]), keys_list['sk'][1])
sum_2_Transaction = Transaction(total_sum_input_1, total_sum_output_1, second_sum_sig, [], 0)

In [119]:
obj1 = sum_1_Transaction.to_dict()
obj2 = sum_2_Transaction.to_dict()
objects = []
objects.append(obj1)
objects.append(obj2)
json_str = json.dumps(objects, indent=4)
with open("InputandOutputSum.json", "w") as outfile:
    outfile.write(json_str)

Transaction Inputs Exist 

In [131]:
ex_input_1 = transaction_input(1, output_5, None)
ex_output_1 = transaction_output(50, keys_list['pubkey'][4].encode(encoder=Base64Encoder).decode(), [])
ex_output_2 = transaction_output(50, keys_list['pubkey'][5].encode(encoder=Base64Encoder).decode(), [])
ex_input_1_list = []
ex_output_1_list = []
ex_input_1_list.append(ex_input_1)
ex_output_1_list.append(ex_output_1)
ex_output_1_list.append(ex_output_2)
ex_sig = createSignature_nonGenesis(json.dumps(([i.to_dict() for i in ex_input_1_list])),json.dumps([o.to_dict() for o in ex_output_1_list]), keys_list['sk'][4])
ex_Transaction_1 = Transaction([], ex_input_1_list, ex_output_1_list, ex_sig, [], 0)

In [132]:
fake_output = transaction_output(72, keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), [])
fake_input = transaction_input(1, fake_output, None)
fake_out_1 = transaction_output(40, keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), [])
fake_out_2 = transaction_output(52, keys_list['pubkey'][2].encode(encoder=Base64Encoder).decode(), [])
fake_input_1_list = []
fake_output_1_list = []
fake_input_1_list.append(fake_input)
fake_output_1_list.append(fake_out_1)
fake_output_1_list.append(fake_out_2)
fake_sig = createSignature_nonGenesis(json.dumps(([i.to_dict() for i in fake_input_1_list])),json.dumps([o.to_dict() for o in fake_output_1_list]), keys_list['sk'][3])
fake_Transaction = Transaction([], fake_input_1_list, fake_output_1_list, fake_sig, [], 0)

In [133]:
object1 = ex_Transaction_1.to_dict()
object2 = fake_Transaction.to_dict()
objects = []
objects.append(object1)
objects.append(object2)
json_str = json.dumps(objects, indent=4)
with open("InputTxNotExist.json", "w") as outfile:
    outfile.write(json_str)

In [ ]:
# True transaction file with 15 different transactions and 8 users all involved 

In [135]:
v_input_1 = transaction_input(1, output_1, None)
v_output_1 = transaction_output(50, keys_list['pubkey'][0].encode(encoder=Base64Encoder).decode(), [])
v_output_2 = transaction_output(50, keys_list['pubkey'][1].encode(encoder=Base64Encoder).decode(), [])
v_input_1_list = []
v_output_1_list = []
v_input_1_list.append(v_input_1)
v_output_1_list.append(v_output_1)
v_output_1_list.append(v_output_2)
v_sig = createSignature_nonGenesis(json.dumps(([i.to_dict() for i in v_input_1_list])),json.dumps([o.to_dict() for o in v_output_1_list]), keys_list['sk'][0])
v_Transaction_1 = Transaction([], v_input_1_list, v_output_1_list, v_sig, [], 0)

In [136]:
v1 = v_Transaction_1.to_dict()
objects = []
objects.append(v1)
json_str = json.dumps(objects, indent=4)
with open("VerifiedTransactions.json", "w") as outfile:
    outfile.write(json_str)